In [43]:
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv


#DATA Cleaning

In [44]:
table = pd.read_csv("data/2021_22_matches", sep=",")
table.tai

,id,season,matchday,date,time,home,guest,result,link
326,108859,2021_22,Spieltag 34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,http://live.2basketballbundesliga.de/g/108859?...
327,108855,2021_22,Spieltag 34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,http://live.2basketballbundesliga.de/g/108855?...
328,108860,2021_22,Spieltag 34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,http://live.2basketballbundesliga.de/g/108860?...
329,108857,2021_22,Spieltag 34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,http://live.2basketballbundesliga.de/g/108857?...
330,108854,2021_22,Spieltag 34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,http://live.2basketballbundesliga.de/g/108854?...


In [45]:
table['matchday'] = table['matchday'].str.replace("Spieltag ","")
table.tail()

,id,season,matchday,date,time,home,guest,result,link
326,108859,2021_22,34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,http://live.2basketballbundesliga.de/g/108859?...
327,108855,2021_22,34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,http://live.2basketballbundesliga.de/g/108855?...
328,108860,2021_22,34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,http://live.2basketballbundesliga.de/g/108860?...
329,108857,2021_22,34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,http://live.2basketballbundesliga.de/g/108857?...
330,108854,2021_22,34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,http://live.2basketballbundesliga.de/g/108854?...


In [40]:
table['matchday'].astype('str')
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        331 non-null    int64 
 1   season    331 non-null    object
 2   matchday  331 non-null    object
 3   date      331 non-null    object
 4   time      297 non-null    object
 5   home      331 non-null    object
 6   guest     331 non-null    object
 7   result    331 non-null    object
 8   link      331 non-null    object
dtypes: int64(1), object(8)
memory usage: 23.4+ KB


# dropping playoff games

In [41]:
table_new = table.drop(table.index[table['matchday'] == 'len(table["matchday"]) > 2'])
table_new.head()

,id,season,matchday,date,time,home,guest,result,link
0,108931,2021_22,PO-1/4 Finale Spiel 1,14.04.2022,19:30,Tigers Tübingen,Eisbären Bremerhaven,82:78,http://live.2basketballbundesliga.de/g/108931?...
1,108930,2021_22,PO-1/4 Finale Spiel 1,22.04.2022,19:30,RÖMERSTROM Gladiators Trier,Bayer Giants Leverkusen,92:68,http://live.2basketballbundesliga.de/g/108930?...
2,108933,2021_22,PO-1/4 Finale Spiel 1,22.04.2022,19:30,Medipolis SC Jena,Uni Baskets Paderborn,89:84,http://live.2basketballbundesliga.de/g/108933?...
3,108932,2021_22,PO-1/4 Finale Spiel 1,26.04.2022,19:30,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,107:103,http://live.2basketballbundesliga.de/g/108932?...
4,108936,2021_22,PO-1/4 Finale Spiel 2,17.04.2022,18:00,Eisbären Bremerhaven,Tigers Tübingen,75:82,http://live.2basketballbundesliga.de/g/108936?...


In [262]:
table_reg_season = table[table['matchday'].map(len) < 3]
table_reg_season.tail()

,id,season,matchday,date,time,home,guest,result,link
326,108859,2021_22,34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,http://live.2basketballbundesliga.de/g/108859?...
327,108855,2021_22,34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,http://live.2basketballbundesliga.de/g/108855?...
328,108860,2021_22,34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,http://live.2basketballbundesliga.de/g/108860?...
329,108857,2021_22,34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,http://live.2basketballbundesliga.de/g/108857?...
330,108854,2021_22,34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,http://live.2basketballbundesliga.de/g/108854?...


In [61]:
table_reg_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 25 to 330
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        306 non-null    int64 
 1   season    306 non-null    object
 2   matchday  306 non-null    object
 3   date      306 non-null    object
 4   time      272 non-null    object
 5   home      306 non-null    object
 6   guest     306 non-null    object
 7   result    306 non-null    object
 8   link      306 non-null    object
dtypes: int64(1), object(8)
memory usage: 23.9+ KB


In [92]:
#dropping date, time, id, link

In [266]:
table_reg_season = table_reg_season.drop('link', axis=1)
table_reg_season.tail()

,id,season,matchday,date,time,home,guest,result,pts+,pts-
326,108859,2021_22,34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,69,101
327,108855,2021_22,34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,121,83
328,108860,2021_22,34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,57,89
329,108857,2021_22,34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,67,84
330,108854,2021_22,34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,73,80


In [93]:
#split result

In [263]:
table_reg_season[['pts+', 'pts-']] = table_reg_season.result.str.split(":", expand=True)
table_reg_season.head()

/var/folders/51/wh5_hyt90lqgvy893hvwhymm0000gn/T/ipykernel_79665/887989402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_reg_season[['pts+', 'pts-']] = table_reg_season.result.str.split(":", expand=True)
/var/folders/51/wh5_hyt90lqgvy893hvwhymm0000gn/T/ipykernel_79665/887989402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_reg_season[['pts+', 'pts-']] = table_reg_season.result.str.split(":", expand=True)


,id,season,matchday,date,time,home,guest,result,link,pts+,pts-
25,108961,2021_22,1,N.N.,NaN,spielfrei,Itzehoe Eagles,--:--,http://live.2basketballbundesliga.de/g/108961?...,--,--
26,108556,2021_22,1,17.09.2021,19:30,RASTA Vechta,ROSTOCK SEAWOLVES,54:68,http://live.2basketballbundesliga.de/g/108556?...,54,68
27,108557,2021_22,1,18.09.2021,18:30,Nürnberg Falcons BC,Artland Dragons,74:71,http://live.2basketballbundesliga.de/g/108557?...,74,71
28,108558,2021_22,1,18.09.2021,19:00,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91,http://live.2basketballbundesliga.de/g/108558?...,74,91
29,108559,2021_22,1,18.09.2021,19:00,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87,http://live.2basketballbundesliga.de/g/108559?...,84,87


In [264]:
table_reg_season.tail()

,id,season,matchday,date,time,home,guest,result,link,pts+,pts-
326,108859,2021_22,34,09.04.2022,19:30,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,http://live.2basketballbundesliga.de/g/108859?...,69,101
327,108855,2021_22,34,09.04.2022,19:30,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,http://live.2basketballbundesliga.de/g/108855?...,121,83
328,108860,2021_22,34,09.04.2022,19:30,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,http://live.2basketballbundesliga.de/g/108860?...,57,89
329,108857,2021_22,34,09.04.2022,19:30,Itzehoe Eagles,RASTA Vechta,67:84,http://live.2basketballbundesliga.de/g/108857?...,67,84
330,108854,2021_22,34,19.04.2022,18:00,ROSTOCK SEAWOLVES,PS Karlsruhe LIONS,73:80,http://live.2basketballbundesliga.de/g/108854?...,73,80


#drop spielfrei

In [86]:
table45 = table_reg_season.drop(table_reg_season.index[table_reg_season['home'] == 'spielfrei'])
table45

,season,matchday,home,guest,result,pts+,pts-
26,2021_22,1,RASTA Vechta,ROSTOCK SEAWOLVES,54:68,54,68
27,2021_22,1,Nürnberg Falcons BC,Artland Dragons,74:71,74,71
28,2021_22,1,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91,74,91
29,2021_22,1,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87,84,87
30,2021_22,1,wiha Panthers Schwenningen,Phoenix Hagen,63:78,63,78
...,...,...,...,...,...,...,...
326,2021_22,34,TEAM EHINGEN URSPRING,RÖMERSTROM Gladiators Trier,69:101,69,101
327,2021_22,34,Medipolis SC Jena,VfL SparkassenStars Bochum,121:83,121,83
328,2021_22,34,VfL Kirchheim Knights,Nürnberg Falcons BC,57:89,57,89
329,2021_22,34,Itzehoe Eagles,RASTA Vechta,67:84,67,84


In [284]:
table_reg_season = table45.drop(table45.index[table45['guest'] == 'spielfrei'])
table_reg_season.head()


,season,matchday,home,guest,result,pts+,pts-
26,2021_22,1,RASTA Vechta,ROSTOCK SEAWOLVES,54:68,54,68
27,2021_22,1,Nürnberg Falcons BC,Artland Dragons,74:71,74,71
28,2021_22,1,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91,74,91
29,2021_22,1,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87,84,87
30,2021_22,1,wiha Panthers Schwenningen,Phoenix Hagen,63:78,63,78


In [285]:
table_reg_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 26 to 330
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    272 non-null    object
 1   matchday  272 non-null    object
 2   home      272 non-null    object
 3   guest     272 non-null    object
 4   result    272 non-null    object
 5   pts+      272 non-null    object
 6   pts-      272 non-null    object
dtypes: object(7)
memory usage: 17.0+ KB


In [287]:
table_reg_season['pts-'] =table_reg_season['pts-'].astype(int)


In [288]:
table_reg_season['pts+'] =table_reg_season['pts+'].astype(int)


In [323]:
table_reg_season['matchday'] =table_reg_season['matchday'].astype(int)

In [290]:
table_reg_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 26 to 330
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    272 non-null    object
 1   matchday  272 non-null    object
 2   home      272 non-null    object
 3   guest     272 non-null    object
 4   result    272 non-null    object
 5   pts+      272 non-null    int64 
 6   pts-      272 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 17.0+ KB


#start the tabellencode

# unique teams

In [291]:
teams = table_reg_season['guest'].unique()
display(teams)

array(['ROSTOCK SEAWOLVES', 'Artland Dragons', 'Medipolis SC Jena',
       'RÖMERSTROM Gladiators Trier', 'Phoenix Hagen', 'Tigers Tübingen',
       'TEAM EHINGEN URSPRING    ', 'PS Karlsruhe LIONS',
       'Itzehoe Eagles', 'Nürnberg Falcons BC', 'Eisbären Bremerhaven',
       'Uni Baskets Paderborn', 'VfL Kirchheim Knights', 'RASTA Vechta',
       'Bayer Giants Leverkusen', 'wiha Panthers Schwenningen',
       'VfL SparkassenStars Bochum'], dtype=object)

In [292]:
tabelle = ['season', 'rank', 'team', 'games_played', 'wins', 'losses', 'points', 'pts_plus', 'pts_minus', 'point_difference' ]
count = 1
season = 2021/22
teams = table_reg_season['guest'].unique()

In [314]:

df = pd.DataFrame(columns=tabelle)
df.head()

,season,rank,team,games_played,wins,losses,points,pts_plus,pts_minus,point_difference


In [276]:
df2 = pd.DataFrame(columns=tabelle)
df2

,season,rank,team,games_played,wins,losses,points,pts_plus,pts_minus,point_difference


#matchday tabelle


In [293]:
matchday_2 = table_reg_season.query('matchday == "2"')
matchday_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 35 to 42
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    8 non-null      object
 1   matchday  8 non-null      object
 2   home      8 non-null      object
 3   guest     8 non-null      object
 4   result    8 non-null      object
 5   pts+      8 non-null      int64 
 6   pts-      8 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 512.0+ bytes


#code , that shows us the team, which is not playing on this gameday


In [271]:
liste_home2 = []
for x in matchday_2['home']:
    liste_home2.append(x)

liste_guest2 = []
for x in matchday_2['guest']:
    liste_guest2.append(x)



set_teams = set(teams)
set_guest2 = set(liste_guest2)
set_home2 = set(liste_home2)
set_spielfrei2 = set_teams.symmetric_difference(set_home2)
set_spielfrei_final2 = set_spielfrei2.symmetric_difference(set_guest2)
spielfrei2 = list(set_spielfrei_final2)

spielfrei2

['Artland Dragons']

#code for the standings only on matchday2

In [294]:

for index, row in matchday_2.iterrows():
    if row['pts+'] > row['pts-']:
        df2.loc[df2.shape[0]]= ['2021_22', 0 , row['home'] , 1 , 1 , 0 , 2 , row['pts+'] , row['pts-'] , (row['pts+'] - row['pts-'])]
        df2.loc[df2.shape[0]] = ['2021_22', 0 , row['guest'] , 1 , 0 , 1 , 0 , row['pts-'] , row['pts+'] , (row['pts-'] - row['pts+'])]
        
    else:
        df2.loc[df2.shape[0]] = ['2021_22', 0 , row['home'] , 1 , 0 , 1 , 0 , row['pts+'] , row['pts-'] , (row['pts+'] - row['pts-'])]
        df2.loc[df2.shape[0]]= ['2021_22', 0 , row['guest'] , 1 , 1 , 0 , 2 , row['pts-'] , row['pts+'] , (row['pts-'] - row['pts+'])]

df2.loc[df2.shape[0]] = ['2021_22', 0 , spielfrei2[0] , 0 , 0 , 0 , 0 , 0 , 0 , 0]

df2.sort_values(['points', 'point_difference', 'pts_plus'], ascending=False)

,season,rank,team,games_played,wins,losses,points,pts_plus,pts_minus,point_difference
12,2021_22,0,PS Karlsruhe LIONS,1,1,0,2,97,70,27
14,2021_22,0,Phoenix Hagen,1,1,0,2,83,57,26
6,2021_22,0,Medipolis SC Jena,1,1,0,2,88,71,17
5,2021_22,0,Itzehoe Eagles,1,1,0,2,94,78,16
10,2021_22,0,RÖMERSTROM Gladiators Trier,1,1,0,2,89,75,14
2,2021_22,0,VfL SparkassenStars Bochum,1,1,0,2,77,65,12
0,2021_22,0,Bayer Giants Leverkusen,1,1,0,2,95,85,10
8,2021_22,0,wiha Panthers Schwenningen,1,1,0,2,83,81,2
16,2021_22,0,Artland Dragons,0,0,0,0,0,0,0
9,2021_22,0,Eisbären Bremerhaven,1,0,1,0,81,83,-2


#mathcday 1 


In [295]:
matchday_1 = (table_reg_season.query('matchday == "1"'))
matchday_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 26 to 33
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    8 non-null      object
 1   matchday  8 non-null      object
 2   home      8 non-null      object
 3   guest     8 non-null      object
 4   result    8 non-null      object
 5   pts+      8 non-null      int64 
 6   pts-      8 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 512.0+ bytes


In [296]:
liste_home = []
for x in matchday_1['home']:
    liste_home.append(x)

liste_guest = []
for x in matchday_1['guest']:
    liste_guest.append(x)




set_guest = set(liste_guest)
set_home = set(liste_home)
set_spielfrei = set_teams.symmetric_difference(set_home)
set_spielfrei_final = set_spielfrei.symmetric_difference(set_guest)
spielfrei = list(set_spielfrei_final)
spielfrei

['Itzehoe Eagles']

#table matchday 1

In [315]:
for index, row in matchday_1.iterrows():
    if row['pts+'] > row['pts-']:
        df.loc[df.shape[0]]= ['2021_22', 0 , row['home'] , 1 , 1 , 0 , 2 , row['pts+'] , row['pts-'] , (row['pts+'] - row['pts-'])]
        df.loc[df.shape[0]] = ['2021_22', 0 , row['guest'] , 1 , 0 , 1 , 0 , row['pts-'] , row['pts+'] , (row['pts-'] - row['pts+'])]
        
    else:
        df.loc[df.shape[0]] = ['2021_22', 0 , row['home'] , 1 , 0 , 1 , 0 , row['pts+'] , row['pts-'] , (row['pts+'] - row['pts-'])]
        df.loc[df.shape[0]]= ['2021_22', 0 , row['guest'] , 1 , 1 , 0 , 2 , row['pts-'] , row['pts+'] , (row['pts-'] - row['pts+'])]

df.loc[df.shape[0]] = ['2021_22', 0 , spielfrei[0] , 0 , 0 , 0 , 0 , 0 , 0 , 0]   
df

,season,rank,team,games_played,wins,losses,points,pts_plus,pts_minus,point_difference
0,2021_22,0,RASTA Vechta,1,0,1,0,54,68,-14
1,2021_22,0,ROSTOCK SEAWOLVES,1,1,0,2,68,54,14
2,2021_22,0,Nürnberg Falcons BC,1,1,0,2,74,71,3
3,2021_22,0,Artland Dragons,1,0,1,0,71,74,-3
4,2021_22,0,VfL SparkassenStars Bochum,1,0,1,0,74,91,-17
5,2021_22,0,Medipolis SC Jena,1,1,0,2,91,74,17
6,2021_22,0,Eisbären Bremerhaven,1,0,1,0,84,87,-3
7,2021_22,0,RÖMERSTROM Gladiators Trier,1,1,0,2,87,84,3
8,2021_22,0,wiha Panthers Schwenningen,1,0,1,0,63,78,-15
9,2021_22,0,Phoenix Hagen,1,1,0,2,78,63,15


In [316]:
df1 = df.sort_values(['points', 'point_difference', 'pts_plus'], ascending=False)
df1

,season,rank,team,games_played,wins,losses,points,pts_plus,pts_minus,point_difference
14,2021_22,0,Bayer Giants Leverkusen,1,1,0,2,109,71,38
12,2021_22,0,Uni Baskets Paderborn,1,1,0,2,98,81,17
5,2021_22,0,Medipolis SC Jena,1,1,0,2,91,74,17
11,2021_22,0,Tigers Tübingen,1,1,0,2,83,67,16
9,2021_22,0,Phoenix Hagen,1,1,0,2,78,63,15
1,2021_22,0,ROSTOCK SEAWOLVES,1,1,0,2,68,54,14
7,2021_22,0,RÖMERSTROM Gladiators Trier,1,1,0,2,87,84,3
2,2021_22,0,Nürnberg Falcons BC,1,1,0,2,74,71,3
16,2021_22,0,Itzehoe Eagles,0,0,0,0,0,0,0
6,2021_22,0,Eisbären Bremerhaven,1,0,1,0,84,87,-3


#indexing and sorting by team for matchdays without standing

In [ ]:
df_sort1 = df1.sort_values('team')

df_sort1 = df_sort1.reset_index()

df_sort1 = df_sort1.drop('index', axis=1)

df_sort1

In [ ]:
df_sort2 = df2.sort_values('team')

df_sort2 = df_sort2.reset_index()

df_sort2 = df_sort2.drop('index', axis=1)

df_sort2

In [332]:
df_result_gd2 = df_sort1 + df_sort2
df_result_gd2['team'] = df_sort2['team']
df_result_gd2['season'] = df_sort2['season']
df_result_gd2

,season,rank,team,games_played,wins,losses,points,pts_plus,pts_minus,point_difference
0,2021_22,0,Artland Dragons,1,0,1,0,71,74,-3
1,2021_22,0,Bayer Giants Leverkusen,2,2,0,4,204,156,48
2,2021_22,0,Eisbären Bremerhaven,2,0,2,0,165,170,-5
3,2021_22,0,Itzehoe Eagles,1,1,0,2,94,78,16
4,2021_22,0,Medipolis SC Jena,2,2,0,4,179,145,34
5,2021_22,0,Nürnberg Falcons BC,2,1,1,2,145,159,-14
6,2021_22,0,PS Karlsruhe LIONS,2,1,1,2,168,179,-11
7,2021_22,0,Phoenix Hagen,2,2,0,4,161,120,41
8,2021_22,0,RASTA Vechta,2,0,2,0,111,151,-40
9,2021_22,0,ROSTOCK SEAWOLVES,2,1,1,2,153,149,4


In [325]:
table_reg_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 26 to 330
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    272 non-null    object
 1   matchday  272 non-null    int64 
 2   home      272 non-null    object
 3   guest     272 non-null    object
 4   result    272 non-null    object
 5   pts+      272 non-null    int64 
 6   pts-      272 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 17.0+ KB


In [333]:
# for schleife für die tabelle

In [354]:
table_reg_season.head(30)

,season,matchday,home,guest,result,pts+,pts-
26,2021_22,1,RASTA Vechta,ROSTOCK SEAWOLVES,54:68,54,68
27,2021_22,1,Nürnberg Falcons BC,Artland Dragons,74:71,74,71
28,2021_22,1,VfL SparkassenStars Bochum,Medipolis SC Jena,74:91,74,91
29,2021_22,1,Eisbären Bremerhaven,RÖMERSTROM Gladiators Trier,84:87,84,87
30,2021_22,1,wiha Panthers Schwenningen,Phoenix Hagen,63:78,63,78
31,2021_22,1,VfL Kirchheim Knights,Tigers Tübingen,67:83,67,83
32,2021_22,1,Uni Baskets Paderborn,TEAM EHINGEN URSPRING,98:81,98,81
33,2021_22,1,Bayer Giants Leverkusen,PS Karlsruhe LIONS,109:71,109,71
35,2021_22,2,Bayer Giants Leverkusen,ROSTOCK SEAWOLVES,95:85,95,85
36,2021_22,2,VfL SparkassenStars Bochum,Tigers Tübingen,77:65,77,65


In [355]:
for i in range(2, 35):
    matchday= table_reg_season.query('matchday == '+str(i)+'')
    

    liste_home = []
    for x in matchday_2['home']:
            liste_home.append(x)

    liste_guest = []
    for x in matchday_2['guest']:
            liste_guest.append(x)



    set_teams = set(teams)
    set_guest = set(liste_guest)
    set_home = set(liste_home)
    set_spielfrei = set_teams.symmetric_difference(set_home)
    set_spielfrei_final = set_spielfrei.symmetric_difference(set_guest)
    spielfrei = list(set_spielfrei_final)
    

    df = pd.DataFrame(columns=tabelle)

    for index, row in matchday.iterrows():
        if row['pts+'] > row['pts-']:
            df.loc[df.shape[0]]= ['2021_22', 0 , row['home'] , 1 , 1 , 0 , 2 , row['pts+'] , row['pts-'] , (row['pts+'] - row['pts-'])]
            df.loc[df.shape[0]] = ['2021_22', 0 , row['guest'] , 1 , 0 , 1 , 0 , row['pts-'] , row['pts+'] , (row['pts-'] - row['pts+'])]
        
        else:
            df.loc[df.shape[0]] = ['2021_22', 0 , row['home'] , 1 , 0 , 1 , 0 , row['pts+'] , row['pts-'] , (row['pts+'] - row['pts-'])]
            df.loc[df.shape[0]]= ['2021_22', 0 , row['guest'] , 1 , 1 , 0 , 2 , row['pts-'] , row['pts+'] , (row['pts-'] - row['pts+'])]

    df.loc[df.shape[0]] = ['2021_22', 0 , spielfrei[0] , 0 , 0 , 0 , 0 , 0 , 0 , 0]   
    df_sort = df.sort_values('team')
    df_sort = df_sort.reset_index()
    df_sort = df_sort.drop('index', axis=1)

    if i == 2:
        df_matchday = df_sort + df_sort1
        df_matchday['team'] = df_sort['team']
        df_matchday['season'] = df_sort['season']
    else:
        #display(df_sort1)
        #df_matchday = df_sort + df_matchday
        #df_matchday['team'] = df_sort['team']
        #df_matchday['season'] = df_sort['season']

#df_main = pd.concat([df_sort1, df_matchday])

    

IndentationError: expected an indented block (3870690388.py, line 49)